In [1]:
# Isabella Sturm
# 2022-01-17
# Super Bowl Analysis
# Web scrape nfl.com to get team stats for data analysis
# Scrape data from pro-football-reference.com for additional statistics

In [2]:
# Import libraries for web scraping
from bs4 import BeautifulSoup
import requests
import pandas as pd

### NFL Data

In [3]:
# Set up starting URL information
# URL = www.nfl.com/stats/team-stats/[type]/[stat]/[year]/reg/all
url_prefix = 'https://www.nfl.com/stats/team-stats/'
url_suffix = '/reg/all'
team_types = ['offense', 'defense', 'special-teams']
off_stats = ['passing', 'rushing', 'receiving', 'scoring', 'downs']
def_stats = off_stats + ['tackles', 'fumbles', 'interceptions']
st_stats = ['field-goals', 'scoring', 'kickoffs', 'punting', 'punt-returns']



In [4]:
# Get all the data from 1970 through 2021
nfl_df = pd.DataFrame()
for year in range(1970,2022):
    curr_year_df = pd.DataFrame()
    # Loop through the different team types and stat types and add to DF
    for team in team_types:
        if team == 'offense':
            for stat in off_stats:
                url = url_prefix + team +'/'+ stat +'/'+ str(year) + url_suffix
                df = pd.read_html(url)[0]
                df = df.add_prefix('off_'+stat+'_')
                if curr_year_df.empty:
                    curr_year_df = df
                else:
                    df.drop('off_'+stat+'_'+'Team', axis=1)
                    curr_year_df = pd.concat([curr_year_df, df], axis=1)
        elif team == 'defense':
            for stat in def_stats:
                url = url_prefix + team +'/'+ stat +'/'+ str(year) + url_suffix
                df = pd.read_html(url)[0]
                df = df.add_prefix('def_'+stat+'_')
                if curr_year_df.empty:
                    curr_year_df = df
                else:
                    df.drop('def_'+stat+'_'+'Team', axis=1)
                    curr_year_df = pd.concat([curr_year_df, df], axis=1)
        else: # special teams
            for stat in st_stats:
                url = url_prefix + team +'/'+ stat +'/'+ str(year) + url_suffix
                df = pd.read_html(url)[0]
                df = df.add_prefix('st_'+stat+'_')
                if curr_year_df.empty:
                    curr_year_df = df
                else:
                    df.drop('st_'+stat+'_'+'Team', axis=1)
                    curr_year_df = pd.concat([curr_year_df, df], axis=1)
                
    # Add current year to dataframe
    curr_year_df['Year'] = year
    # Append current year df to main nfl dataframe
    nfl_df = nfl_df.append(curr_year_df)
    

In [5]:
print(nfl_df.shape)

(1528, 178)


In [6]:
# Export data to csv for future (nfl years available changes yearly)
nfl_df.to_csv('nfl_stats_1970_2021.csv')

### Pro Football Reference Data

In [7]:
# Set up starting URL information
# URL = pro-football-reference.com/teams/[team]/[year]/gamelogs
url_prefix = 'https://www.pro-football-reference.com/teams/'
url_suffix = '.htm'
teams = {'buf': 'Bills','nwe': 'Patriots',
         'mia': 'Dolphins','nyj': 'Jets',
         'oti': 'Titans','clt': 'Colts',
         'htx': 'Texans','jax': 'Jaguars',
         'cin': 'Bengals','pit': 'Steelers',
         'cle': 'Browns','rav': 'Ravens',
         'kan': 'Chiefs','rai': 'Raiders',
         'sdg': 'Chargers','den': 'Broncos',
         'dal': 'Cowboys','phi': 'Eagles',
         'was': 'Football Team','nyg': 'Giants',
         'tam': 'Buccaneers','nor': 'Saints',
         'atl': 'Falcons','car': 'Panthers',
         'gnb': 'Packers','min': 'Vikings',
         'chi': 'Bears','det': 'Lions',
         'ram': 'Rams','crd': 'Cardinals',
         'sfo': '49ers','sea': 'Seahawks'}


In [8]:
# Get all the data from 1976 through 2021
pfr_df = pd.DataFrame()
for year in range(1976,2022):
    curr_year_df = pd.DataFrame()
    
    for team in teams:
        url = url_prefix + team +'/'+ str(year) + url_suffix
        # The first table is Team Stats and Rankings
        # Second is schedule and game results
        try:
            df = pd.read_html(url, header=1)[1]
            df['Team'] = teams[team]
            # Check if team was in the super bowl
            exists = 'SuperBowl' in df['Week']
            df['SuperBowl'] = exists
            # If the team was in Super Bowl, check if they won
            isWinner = False
            if exists:
                isWinner = 'W' in df[df['Week'] == 'SuperBowl']['Unnamed: 5']
            df['SuperBowlWinner'] = isWinner
            curr_year_df = curr_year_df.append(df)
        except:
            print('No data for team {} in year {}'.format(team, year))
        
    curr_year_df['Year'] = year
    pfr_df = pfr_df.append(curr_year_df)
    

No data for team htx in year 1976
No data for team jax in year 1976
No data for team rav in year 1976
No data for team car in year 1976
No data for team htx in year 1977
No data for team jax in year 1977
No data for team rav in year 1977
No data for team car in year 1977
No data for team htx in year 1978
No data for team jax in year 1978
No data for team rav in year 1978
No data for team car in year 1978
No data for team htx in year 1979
No data for team jax in year 1979
No data for team rav in year 1979
No data for team car in year 1979
No data for team htx in year 1980
No data for team jax in year 1980
No data for team rav in year 1980
No data for team car in year 1980
No data for team htx in year 1981
No data for team jax in year 1981
No data for team rav in year 1981
No data for team car in year 1981
No data for team htx in year 1982
No data for team jax in year 1982
No data for team rav in year 1982
No data for team car in year 1982
No data for team htx in year 1983
No data for te

In [9]:
print(pfr_df.shape)

(24360, 29)


In [10]:
# Export data to csv for future
pfr_df.to_csv('pfr_stats_1976_2021.csv')

In [11]:
pfr_df.head()

,Week,Day,Date,Unnamed: 3,Unnamed: 4,Unnamed: 5,OT,Rec,Unnamed: 8,Opp,...,PassY.1,RushY.1,TO.1,Team,SuperBowl,SuperBowlWinner,Year,Offense,Defense,Sp. Tms
0,1,Mon,September 13,9:00PM ET,boxscore,L,NaN,0-1,NaN,Miami Dolphins,...,197.0,204.0,3.0,Bills,False,False,1976,NaN,NaN,NaN
1,2,Sun,September 19,1:00PM ET,boxscore,L,NaN,0-2,NaN,Houston Oilers,...,52.0,208.0,1.0,Bills,False,False,1976,NaN,NaN,NaN
2,3,Sun,September 26,1:00PM ET,boxscore,W,NaN,1-2,@,Tampa Bay Buccaneers,...,168.0,170.0,2.0,Bills,False,False,1976,NaN,NaN,NaN
3,4,Sun,October 3,1:00PM ET,boxscore,W,NaN,2-2,NaN,Kansas City Chiefs,...,341.0,161.0,6.0,Bills,False,False,1976,NaN,NaN,NaN
4,5,Sun,October 10,1:00PM ET,boxscore,L,NaN,2-3,@,New York Jets,...,63.0,207.0,4.0,Bills,False,False,1976,NaN,NaN,NaN


In [12]:
pfr_df.tail()

,Week,Day,Date,Unnamed: 3,Unnamed: 4,Unnamed: 5,OT,Rec,Unnamed: 8,Opp,...,PassY.1,RushY.1,TO.1,Team,SuperBowl,SuperBowlWinner,Year,Offense,Defense,Sp. Tms
13,14,Sun,December 12,1:00PM ET,boxscore,W,NaN,5-8,@,Houston Texans,...,317.0,63.0,NaN,Seahawks,False,False,2021,21.71,2.60,-9.75
14,15,Tue,December 21,7:00PM ET,boxscore,L,NaN,5-9,@,Los Angeles Rams,...,209.0,123.0,1.0,Seahawks,False,False,2021,-8.13,-4.19,3.72
15,16,Sun,December 26,4:05PM ET,boxscore,L,NaN,5-10,NaN,Chicago Bears,...,229.0,88.0,NaN,Seahawks,False,False,2021,8.31,-7.60,-3.95
16,17,Sun,January 2,4:25PM ET,boxscore,W,NaN,6-10,NaN,Detroit Lions,...,262.0,95.0,3.0,Seahawks,False,False,2021,32.32,-3.55,-2.68
17,18,Sun,January 9,4:25PM ET,boxscore,W,NaN,7-10,@,Arizona Cardinals,...,187.0,118.0,1.0,Seahawks,False,False,2021,12.83,-0.21,-1.02
